In [2]:
import pandas as pd
import re
from abctool import abctool, abctune
import gc

In [3]:
df = pd.read_csv('/home/devcontainers/git/Tune_Similarity/notebooks/k_means_cluster/data/cleaned_data.csv')
df.head()

,Unnamed: 0,tune_id,setting_id,name,type,meter,mode,abc,date,username,full_abc
0,0,18105,35234,"$150 Boot, The",polka,2/4,Gmajor,|:d>g fe|dB AG|E/F/G E/F/G|BA GF|\nd>g fe|dB A...,2019-07-06 04:39:09,NfldWhistler,"X:35234\nT:$150 Boot, The\nM:2/4\nK:Gmajor\n|:..."
1,1,11931,11931,'G Iomain Nan Gamhna,slip jig,9/8,Gmajor,dBB B2 A BAG|dBB Bcd efg|dBB B2 A BAG|eAA dBG ...,2012-05-17 07:49:26,iliketurtles,X:11931\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Gmaj...
2,2,11931,48772,'G Iomain Nan Gamhna,slip jig,9/8,Amixolydian,|:dBB BBA BAG|dBB Bcd efg|dBB BBA BAG|e2A dBG ...,2023-11-25 22:54:00,birlibirdie,X:48772\nT:'G Iomain Nan Gamhna\nM:9/8\nK:Amix...
3,3,15326,28560,'S Ann An Ìle,strathspey,4/4,Gmajor,|:G>A B>G c>A B>G|E<E A>G F<D D2|G>A B>G c>A B...,2016-03-31 15:34:45,danninagh,X:28560\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\n|:G...
4,4,15326,28582,'S Ann An Ìle,strathspey,4/4,Gmajor,"uD2|:{F}v[G,2G2]uB>ud c>A B>G|{D}E2 uA>uG F<D ...",2016-04-03 09:15:08,DonaldK,X:28582\nT:'S Ann An Ìle\nM:4/4\nK:Gmajor\nuD2...


In [7]:
def get_transpose_distance(m):
    # m - mode of the tune
    # returns distance to transpose so that tonic is C
    # proper way to do this would be to do regex to match any # or b, but we don't have those keys in the dataset
    tonic = m[0]
    # again, being lazy
    distance = {
        "A" : -9,
        "B" : -11,
        "C" : 0,
        "D" : -2,
        "E" : -4,
        "F" : -5,
        "G" : -7
    }
    return distance[tonic]



def transpose_row(row):
    m = row["mode"]
    abc = row["full_abc"]
    tune = abctune(abc)
    print(f"[{row.name}] PARSED")
    d = get_transpose_distance(m)
    tune.transpose(d)
    print(f"[{row.name}] TRANSPOSED")
    abc_out = tune.getAbc()
    del tune
    return abc_out




In [8]:
# df["transposed_abc"] = df.apply(transpose_row, axis = 1)

def safe_transpose_row(row):
    try:
        return transpose_row(row)
    except Exception as e:
        print(f"Error on row {row.name}: {e}")
        return None



In [9]:
results = []
for i, row in df.iterrows():
    try:
        result = safe_transpose_row(row)
        results.append(result)
    except Exception as e:
        print(f"❌ Row {i} failed: {e}")
        results.append(None)

    if i % 1000 == 0:
        print(f"✅ Processed {i} rows")
        gc.collect()

df["transposed_abc"] = results

[0] PARSED
[0] TRANSPOSED
✅ Processed 0 rows
[1] PARSED
[1] TRANSPOSED
[2] PARSED
[2] TRANSPOSED
[3] PARSED
[3] TRANSPOSED
[4] PARSED
[4] TRANSPOSED
[5] PARSED
[5] TRANSPOSED
[6] PARSED
[6] TRANSPOSED
[7] PARSED
[7] TRANSPOSED
[8] PARSED
[8] TRANSPOSED
[9] PARSED
[9] TRANSPOSED
[10] PARSED
[10] TRANSPOSED
[11] PARSED
[11] TRANSPOSED
[12] PARSED
[12] TRANSPOSED
[13] PARSED
[13] TRANSPOSED
[14] PARSED
[14] TRANSPOSED
[15] PARSED
[15] TRANSPOSED
[16] PARSED
[16] TRANSPOSED
[17] PARSED
[17] TRANSPOSED
[18] PARSED
[18] TRANSPOSED
[19] PARSED
[19] TRANSPOSED
[20] PARSED
[20] TRANSPOSED
[21] PARSED
[21] TRANSPOSED
[22] PARSED
[22] TRANSPOSED
[23] PARSED
[23] TRANSPOSED
[24] PARSED
[24] TRANSPOSED
[25] PARSED
[25] TRANSPOSED
[26] PARSED
[26] TRANSPOSED
[27] PARSED
[27] TRANSPOSED
[28] PARSED
[28] TRANSPOSED
[29] PARSED
[29] TRANSPOSED
[30] PARSED
[30] TRANSPOSED
[31] PARSED
[31] TRANSPOSED
[32] PARSED
[32] TRANSPOSED
[33] PARSED
[33] TRANSPOSED
[34] PARSED
[34] TRANSPOSED
[35] PARSED
[35] TRAN

In [10]:
df.to_csv('data/transposed_tunes.csv')